# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 65 new papers today
          14 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/13 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.07353


extracting tarball to tmp_2303.07353... done.
Retrieving document from  https://arxiv.org/e-print/2303.07376


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.07353/skeleton.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:361: LatexWarning: Latex injecting: 'journals_mnras' from 'tmp_2303.07353/journals_mnras.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2303.07376... done.
Retrieving document from  https://arxiv.org/e-print/2303.07387


...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.08088


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.07943/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2303.08088...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.08088/PaperDraft.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.07410-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.07410) | **Relative Alignment Between Magnetic Fields and Molecular Gas Structure  in Molecular Clouds**  |
|| Renato Mazzei, et al. -- incl., <mark>Zhi-Yun Li</mark> |
|*Appeared on*| *2023-03-15*|
|*Comments*| *18 pages, 13 figures, accepted for publication in MNRAS*|
|**Abstract**| We compare the structure of synthetic dust polarization with synthetic molecular line emission from radiative transfer calculations using a 3-dimensional, turbulent collapsing-cloud magnetohydrodynamics simulation. The histogram of relative orientations (HRO) technique and the projected Rayleigh statistic (PRS) are considered. In our trans-Alfv\'enic (more strongly magnetized) simulation, there is a transition to perpendicular alignment at densities above $\sim$$4 \times 10^{3}$ cm$^{-3}$. This transition is recovered in most of our synthetic observations of optically thin molecular tracers, however for $^{12}$CO it does not occur and the PRS remains in parallel alignment across the whole observer-space. We calculate the physical depth of the optical depth $\tau = 1$ surface and find that for $^{12}$CO it is largely located in front of the cloud midplane, suggesting that $^{12}$CO is too optically thick and instead mainly probes low volume density gas. In our super-Alfv\'enic simulation, the magnetic field becomes significantly more tangled, and all observed tracers tend toward no preference for perpendicular or parallel alignment. An observable difference in alignment between optically thin and optically thick tracers may indicate the presence of a dynamically important magnetic field, though there is some degeneracy with viewing angle. We convolve our data with a Gaussian beam and compare it with HRO results of the Vela C molecular cloud. We find good agreement between these results and our sub-Alfv\'enic simulations when viewed with the magnetic field in the plane-of-the-sky (especially when sensitivity limitations are considered), though the observations are also consistent with an intermediately inclined magnetic field. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.07353-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.07353) | **Minor Flares on Cygnus X-3 -- VLBI Prospects**  |
|| Ralph E. Spencer, Justin D. Bray, <mark>David A. Green</mark>, Michael A. Garrett |
|*Appeared on*| *2023-03-15*|
|*Comments*| *Accepted to PoS (EVN2022): proceedings of 15th European VLBI Network Symposium*|
|**Abstract**| The cm-wavelength radio flares on Cygnus X-3 have been studied for many years. Our recent paper (Spencer et al., 2022) looked again at the minor flares (flux density S of a few 100 mJy) and compared their properties with those of a sample of major flares (S > 1 Jy). We find that the minor flares have rise times and duration of ~ 1 hour, as opposed to ~ days for the major flares. Minor flares show more rapid expansion of the synchrotron radiation emitting material than in the strong flares. They also appear closer to the binary, whereas the large flares form a more developed jet, i.e. the jets formed in minor flares are short and wide, those in major flares are long and thin. We used the results of Fender & Bright (2019) to calculate the magnetic field and expansion velocity as a fraction B of the speed of light under minimum energy conditions when the source is optically thick for samples of minor and major flares. The minimum power in the source was found using the rise time of the flares. The minor flares have lower minimum power but have larger velocities and energy densities than the major flares. Minor flares can occur while a major flare is in progress, suggesting an indirect coupling between them. The spectral evolution of the minor flares can be explained by either an expanding synchrotron source or a shock model. Further investigation requires high resolution VLBI observations at the 1 mas level if we wish to understand the development of the source. The problem is that Cygnus X-3 is strongly scattered by the interstellar medium so high frequencies in the several 10s of GHz are required for the resolution needed. The minor flares are rapid [...] and hence only short snapshot VLBI observations can capture the structure. Large numbers of telescopes are required which is a problem at the highest frequencies. We discuss the VLBI possibilities and trade-offs for this awkward object. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.07376-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.07376) | **Metal Factories in the Early Universe**  |
|| Stephen Eales, et al. -- incl., <mark>Matthew W.L. Smith</mark> |
|*Appeared on*| *2023-03-15*|
|*Comments*| *Submitted to MNRAS*|
|**Abstract**| We have measured the mass of metals in 13 submillimetre galaxies at z~4 in which the gas, based on previous observations, lies in a cold rotating disk. We measured the metal masses using either the submillimetre line or continuum emission from three tracers of the overall metal content - carbon atoms, carbon monoxide molecules and dust grains - using the first calibration of this technique that treats all three tracers simultaneously (Dunne et al. 2022). We obtain very similar mass estimates from the different tracers, which are similar to the entire metal content of a present-day massive early-type galaxy. We used the dynamical masses of these galaxies to set an upper limit on the mass of the interstellar medium in each galaxy, allowing us to set a lower limit on the metal abundance of the ISM, finding values for many of the galaxies well above the solar value. The only caveat to this result is that we cannot be certain for all galaxies that the emission from the tracer comes from within the region covered by the dynamical analysis. Our high values for the metal abundance are supported by a recent JWST study of one galaxy which found a super-solar metal abundance approximately 3 times greater than our lower limit. Using two chemical evolution models, we show that the high metal masses and metal abundances are what is expected shortly after the formation of a galaxy for a top-heavy IMF. We suggest a scenario for galaxy evolution in which massive galaxies reach a high metal abundance during their formation phase, which is then gradually reduced by dry mergers with lower mass galaxies. We use the chemical-evolution models to show that the metals in the outflows from massive early-type galaxies in their formation phase can quantitatively explain the long-standing puzzle that approximately 75% of the metals in clusters of galaxies is in the intracluster gas rather than in the galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.07619-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.07619) | **Observational Evidence for Large-Scale Gas Heating in a Galaxy  Protocluster at z=2.30**  |
|| Chenze Dong, et al. -- incl., <mark>Khee-Gan Lee</mark> |
|*Appeared on*| *2023-03-15*|
|*Comments*| *12 pages, 5 figures (including one interactive figure at this https URL). Accepted by ApJL*|
|**Abstract**| We report a $z=2.30$ galaxy protocluster (COSTCO-I) in the COSMOS field, where the Lyman-$\alpha$ forest as seen in the CLAMATO IGM tomography survey does not show significant absorption. This departs from the transmission-density relationship (often dubbed the fluctuating Gunn-Peterson approximation; FGPA) usually expected to hold at this epoch, which would lead one to predict strong Ly$\alpha$ absorption at the overdensity. For comparison, we generate mock Lyman-$\alpha$ forest maps by applying FGPA to constrained simulations of the COSMOS density field, and create mocks that incorporate the effects of finite sightline sampling, pixel noise, and Wiener filtering. Averaged over $r=15\,h^{-1}\,\mathrm{Mpc}$ around the protocluster, the observed Lyman-$\alpha$ forest is consistently more transparent in the real data than in the mocks, indicating a rejection of the null hypothesis that the gas in COSTCO-I follows FGPA ($p=0.0026$, or $2.79 \sigma$ significance). It suggests that the large-scale gas associated with COSTCO-I is being heated above the expectations of FGPA, which might be due to either large-scale AGN jet feedback or early gravitational shock heating. COSTCO-I is the first known large-scale region of the IGM that is observed to be transitioning from the optically-thin photoionized regime at Cosmic Noon, to eventually coalesce into an intra-cluster medium (ICM) by $z=0$. Future observations of similar structures will shed light on the growth of the ICM and allow constraints on AGN feedback mechanisms. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.07721-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.07721) | **Photometry and spectroscopy of the Type Icn supernova 2021ckj: The  diverse properties of the ejecta and circumstellar matter of Type Icn SNe**  |
|| T. Nagao, et al. -- incl., <mark>T. E. Müller-Bravo</mark> |
|*Appeared on*| *2023-03-15*|
|*Comments*| *13 pages, 5 figures, accepted for publication in A&A*|
|**Abstract**| We present photometric and spectroscopic observations of the Type Icn supernova (SN) 2021ckj. Spectral modeling of SN 2021ckj reveals that its composition is dominated by oxygen, carbon and iron group elements, and the photospheric velocity at peak is ~10000 km/s. From the light curve (LC) modeling applied to SNe 2021ckj, 2019hgp, and 2021csp, we find that the ejecta and CSM properties of Type Icn SNe are diverse. SNe 2021ckj and 2021csp likely have two ejecta components (an aspherical high-energy component and a spherical standard-energy component) with a roughly spherical CSM, while SN 2019hgp can be explained by a spherical ejecta-CSM interaction alone. The ejecta of SNe 2021ckj and 2021csp have larger energy per ejecta mass than the ejecta of SN 2019hgp. The density distribution of the CSM is similar in these three SNe, and is comparable to those of Type Ibn SNe. This may imply that the mass-loss mechanism is common between Type Icn (and also Type Ibn) SNe. The CSM masses of SN 2021ckj and SN 2021csp are higher than that of SN 2019hgp, although all these values are within the diversity seen in Type Ibn SNe. The early spectrum of SN 2021ckj shows narrow emission lines from C II and C III, without a clear absorption component, in contrast with that observed in SN 2021csp. The similarity of the emission components of these lines implies that the emitting regions of SNe 2021ckj and 2021csp have similar ionization states, and thus suggests that they have similar properties of the ejecta and CSM, which is inferred also from the LC modeling. Taking into account the difference in the strength of the absorption features, this heterogeneity may be attributed to viewing angle effects in otherwise common aspherical ejecta. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.07832-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.07832) | **First Detailed Analysis of a Relatively Deep, Low Mass-ratio Contact  Binary: ATO J108.6991+27.8306**  |
|| Shuo Ma, et al. -- incl., <mark>Yu Zhang</mark>, <mark>Chenyang He</mark> |
|*Appeared on*| *2023-03-15*|
|*Comments*| *7 pages, 6 figures*|
|**Abstract**| We present the first detailed photometric analysis of ATO J108.6991+27.8306 (hereinafter as J108). The short-period close binary J108 was observed by the Nanshan 1 m Wide Field Telescope of the Xinjiang Astronomical Observatory. The obtained BVRI-band light curves were used to determine the photometric solution by using the 2003 version of the Wilson-Devinney code. J108 is a typical deep ( f > 50%), low mass ratio (q < 0.25) overcontact binary system with a mass ratio of q = 0.1501 and a fill-out factor of f = 50.1 %, suggesting that it is in the late evolutionary stage of contact binary systems. We found the target to be a W-type W UMa binary and provided evidence for the presence of starspots on both components. From the temperature-luminosity diagram, the main component is the evolved main sequence star with an evolutionary age of about 7.94 Gyr. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.07943-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.07943) | **SKA Science Data Challenge 2: analysis and results**  |
|| P. Hartley, et al. -- incl., <mark>L. Smith</mark>, <mark>L. Zhang</mark>, <mark>X. Zhang</mark>, <mark>Y. Zhang</mark> |
|*Appeared on*| *2023-03-15*|
|*Comments*| *Under review by MNRAS; 28 pages, 16 figures*|
|**Abstract**| The Square Kilometre Array Observatory (SKAO) will explore the radio sky to new depths in order to conduct transformational science. SKAO data products made available to astronomers will be correspondingly large and complex, requiring the application of advanced analysis techniques to extract key science findings. To this end, SKAO is conducting a series of Science Data Challenges, each designed to familiarise the scientific community with SKAO data and to drive the development of new analysis techniques. We present the results from Science Data Challenge 2 (SDC2), which invited participants to find and characterise 233245 neutral hydrogen (Hi) sources in a simulated data product representing a 2000~h SKA MID spectral line observation from redshifts 0.25 to 0.5. Through the generous support of eight international supercomputing facilities, participants were able to undertake the Challenge using dedicated computational resources. Alongside the main challenge, `reproducibility awards' were made in recognition of those pipelines which demonstrated Open Science best practice. The Challenge saw over 100 participants develop a range of new and existing techniques, with results that highlight the strengths of multidisciplinary and collaborative effort. The winning strategy -- which combined predictions from two independent machine learning techniques to yield a 20 percent improvement in overall performance -- underscores one of the main Challenge outcomes: that of method complementarity. It is likely that the combination of methods in a so-called ensemble approach will be key to exploiting very large astronomical datasets. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.08088-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.08088) | **How Cosmic Web Environment Affects Galaxy Quenching Across Cosmic Time**  |
|| Farhanul Hasan, et al. -- incl., <mark>Joseph N. Burchett</mark> |
|*Appeared on*| *2023-03-15*|
|*Comments*| *5 Figures, 15 pages, submitted to ApJ Letters*|
|**Abstract**| We investigate how cosmic web structures affect galaxy quenching in the IllustrisTNG (TNG-100) cosmological simulations by reconstructing the cosmic web in each snapshot using the DisPerSE framework. We measure the distance from each galaxy with stellar mass log(M*/Msun)>=8 to the nearest node (dnode) and the nearest filament spine (dfil) and study the dependence of both median specific star formation rate (<sSFR>) and median gas fraction (<fgas>) on these distances. We find that <sSFR> of galaxies is only dependent on cosmic web environment at z<2, with the dependence increasing with time. At z<=0.5, 8<=log(M*/Msun)<9 galaxies are quenched at dnode<1 Mpc, and significantly star formation-suppressed at dfil<1 Mpc, trends which are driven mostly by satellite galaxies. At z<=1, in contrast to the monotonic rise in <sSFR> of log(M*/Msun)<10 galaxies with dnode and dfil, log(M*/Msun)>=10 galaxies actually experience an upturn in <sSFR> at dnode<0.2 Mpc (this is caused by both satellites and centrals). Much of this cosmic web-dependence of star formation activity can be explained by the evolution in <fgas>. Our results suggest that in the past ~10 Gyr, low-mass satellites are quenched by rapid gas stripping in dense environments near nodes and gradual gas starvation in intermediate-density environments near filaments, while at earlier times cosmic web structures efficiently channeled cold gas into most galaxies. State-of-the-art ongoing spectroscopic surveys such as SDSS and DESI, as well as those planned with JWST and Roman are required to test our predictions against observations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

196  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  "7 Days" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
